In [1]:
import json
import os

In [2]:
data = []
with open('prepared-hansard-llama3.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if len(l['text']) < 100:
            continue
        data.append(l['text'])

In [3]:
len(data)

140927

In [4]:
# !rm -rf rephrase-hansard-llama3
!mkdir rephrase-hansard-llama3

mkdir: cannot create directory ‘rephrase-hansard-llama3’: File exists


In [5]:
results = []
for i in range(len(data)):
    
    s = """
```
{{replace_me}}
```

restructure malaysia hansard text above to become a complete malay wikipedia markdown page, start it with [START] and end with [END]
    """.strip()
    s = s.replace('{{replace_me}}', data[i])
    results.append(s)
        
len(results)

140927

In [6]:
from openai import OpenAI
import requests

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

def answer(q, i):
    filename = f'rephrase-hansard-llama3/{i}.json'
    if os.path.exists(filename):
        return
    
    results = []
    openai = OpenAI(
        base_url='https://llama-3.us.mesolitica.com/v1',
        api_key='empty',
    )
    for _ in range(1):
        
        while True:
            try:
                
                json_data = {
                    'messages': [
                        {
                            'role': 'user',
                            'content': q,
                        },
                    ],
                    'model': 'model',
                    'stop': [
                        '<|eot_id|>',
                    ],
                    'temperature': 0.9,
                    'max_tokens': 8192,
                }
                response = requests.post(
                    'https://llama-3.us.mesolitica.com/v1/chat/completions', 
                    headers=headers, json=json_data, timeout = 60 * 10)
                
                r = response.json()['choices'][0]['message']['content']
                results.append(r)
                break
            except Exception as e:
                # print(e)
                pass
                
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)

In [7]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [8]:
urls = [(q, no) for no, q in enumerate(results)]

In [9]:
answer(*urls[10005])

In [10]:
urls[10005]

("```\nJadi saya berharap Harakah pun tidak tulis dalam tulisan jawi. [Disorak] Itu kata orang yang hendak membela tulisan jawi [Ketawa]. Dahulu marah dengan bahasa Inggeris malah sebahagian daripada Harakah juga ditulis dalam bahasa Inggeris. Jadi lain cakap, lain bikin. Inilah PAS [Ketawa] [Dewan riuh] Jadi saya berharap, Tuan Yang di-Pertua, jangan kita ungkit dan jangan kita mengata-ngatai bahawa kita tidak melakukan ini. Saya berharap pihak PAS bersama dengan kerajaan boleh kita usaha memperbaiki di mana ada kelemahan, mari kita buat cadangan. Kita sila dengar. Saya tahu Ahli Yang Berhormat bagi Parit Buntar berkali-kali mengatakan mengenai tulisan jawi. Langkah-langkah kita sudah ambil dengan secara serius. Kita bukan main-main. Begitu juga soal SAR dan tulisan jawi ini adalah berbeza. Jangan kata lulusan daripada SAR boleh tulisan jawi lebih baik daripada sekolah kebangsaan. Semuanya kita kata kalau usaha itu kita lakukan anak-anak kita tetap boleh dalam tulisan jawi dan Al- Qur

In [11]:
with open('rephrase-hansard-llama3/10005.json') as fopen:
    t = json.load(fopen)
    
print(t[0])

Here is the restructured text in Malay Wikipedia markdown format:

[START]

**Debat Parlimen**

**Tulisan Jawi**

Dalam satu sesi debat di Parlimen, Ahli Parlimen Bagan Datuk, Dato' Mahadzir bin Mohd. Khir, membahas mengenai isu tulisan jawi.

#### Ucapan Dato' Mahadzir bin Mohd. Khir

Jadi saya berharap Harakah pun tidak tulis dalam tulisan jawi. [Disorak] Itu kata orang yang hendak membela tulisan jawi [Ketawa]. Dahulu marah dengan bahasa Inggeris malah sebahagian daripada Harakah juga ditulis dalam bahasa Inggeris. Jadi lain cakap, lain bikin. Inilah PAS [Ketawa] [Dewan riuh] Jadi saya berharap, Tuan Yang di-Pertua, jangan kita ungkit dan jangan kita mengata-ngatai bahawa kita tidak melakukan ini. Saya berharap pihak PAS bersama dengan kerajaan boleh kita usaha memperbaiki di mana ada kelemahan, mari kita buat cadangan. Kita sila dengar. Saya tahu Ahli Yang Berhormat bagi Parit Buntar berkali-kali mengatakan mengenai tulisan jawi. Langkah-langkah kita sudah ambil dengan secara seriu

In [12]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

 21%|██████████████▎                                                     | 29773/140927 [4:53:35<337:55:55, 10.94s/it]